In [1]:
import socketio
import eventlet
import numpy as np
from flask import Flask
from keras.models import load_model
import base64
from io import BytesIO
from PIL import Image
import cv2

In [2]:
#https://python-socketio.readthedocs.io/en/latest/server.html#eventlet
#sio = socketio.Server(async_mode='eventlet')

sio = socketio.Server()

app = Flask(__name__) #'__main__'
speed_limit = 10
def img_preprocess(img):
    img = img[60:135,:,:]
    img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    img = cv2.GaussianBlur(img, (3, 3), 0)
    img = cv2.resize(img, (200, 66))
    img = img/255
    return img

In [3]:
@sio.on('connect')
def connect(sid, environ):
    print('Connected')
    send_control(0, 0)

def send_control(steering_angle, throttle):
    sio.emit('steer', data = {
        'steering_angle': steering_angle.__str__(),
        'throttle': throttle.__str__()
    })

In [4]:
model = load_model('pista2_center_left_right_model_v1-4.h5')

In [5]:
@sio.on('telemetry')
def telemetry(sid, data):
    speed = float(data['speed'])
    image = Image.open(BytesIO(base64.b64decode(data['image'])))
    image = np.asarray(image)
    image = img_preprocess(image)
    image = np.array([image])
    steering_angle = float(model.predict(image))
    throttle = 1.0 - speed/speed_limit
    print('{} {} {}'.format(steering_angle, throttle, speed))
    send_control(steering_angle, throttle)

In [ ]:
#app = socketio.WSGIApp(sio)
app = socketio.Middleware(sio, app)
eventlet.wsgi.server(eventlet.listen(('', 4567)), app)

(5456) wsgi starting up on http://0.0.0.0:4567
(5456) accepted ('127.0.0.1', 53752)


Connected
1/1 [==============================] - 2s 2s/step
0.3142637610435486 0.8686 1.314
1/1 [==============================] - 0s 18ms/step
0.3142637610435486 0.8248 1.752
1/1 [==============================] - 0s 19ms/step
0.3142637610435486 0.8248 1.752
1/1 [==============================] - 0s 19ms/step
0.24328774213790894 0.99999 0.0001
1/1 [==============================] - 0s 17ms/step
0.24445368349552155 0.944 0.56
1/1 [==============================] - 0s 19ms/step
0.26163560152053833 0.8933 1.067
1/1 [==============================] - 0s 18ms/step
0.313891738653183 0.8587899999999999 1.4121
1/1 [==============================] - 0s 19ms/step
0.2843407392501831 0.79616 2.0384
1/1 [==============================] - 0s 18ms/step
0.27682268619537354 0.73454 2.6546
1/1 [==============================] - 0s 19ms/step
0.2751575708389282 0.69669 3.0331
1/1 [==============================] - 0s 17ms/step
0.2071579545736313 0.64132 3.5868
1/1 [==============================] - 0s 19